# 03: Entrenamiento del Modelo para Ayatsil (v1)

* **Autor:** Tu Nombre
* **Fecha de Versión:** 2025-07-19
* **Versión:** 1.0 (Simple)
* **Proyecto:** Pronóstico de Producción - CNH

---

## Objetivo
Este notebook carga los datos con características simples de Ayatsil, entrena un modelo base XGBoost, lo evalúa y guarda los artefactos resultantes (modelo y evaluación).

## Entradas
* `gold/ayatsil_features_simple_v1.parquet`

## Salidas
* `ml/modelos/ayatsil_xgb_simple_v1.json`
* `ml/evaluacion/eval_ayatsil_simple_v1.json`

In [35]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import json
import os

In [37]:
# 0. Definir rutas del proyecto
ruta_base = os.path.abspath(os.path.join(os.getcwd(), "..",".."))
ruta_gold = os.path.join(ruta_base, "gold")
ruta_modelos = os.path.join(ruta_base, "ml", "modelos")
ruta_evaluacion = os.path.join(ruta_base, "ml", "evaluacion")

In [39]:
# 1. Cargar el dataset con características simples de AYATSIL
df_modelo = pd.read_parquet(os.path.join(ruta_gold, "ayatsil_features_for_training.parquet"))

In [41]:
# 2. Definir Características (X) y Objetivo (y) - VERSIÓN SIMPLE
features = [
        'produccion_mes_anterior',
        'produccion_promedio_3m',
        'mes',
        'trimestre'
    ]
target = 'petroleo_mbd'

X = df_modelo[features]
y = df_modelo[target]
print("\nCaracterísticas (X) y objetivo (y) definidos para Ayatsil (versión simple).")



Características (X) y objetivo (y) definidos para Ayatsil (versión simple).


In [43]:
# 3. Dividir los datos en Entrenamiento y Prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [45]:
# 4. Entrenar el Modelo XGBoost
model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=1000,
        learning_rate=0.05,
        early_stopping_rounds=5
    )
    
model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=False
    )
print("\nModelo para Ayatsil (versión simple) entrenado con éxito.")


Modelo para Ayatsil (versión simple) entrenado con éxito.


In [47]:
# 5. Evaluar el Modelo
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Evaluación completada. RMSE: {rmse:.2f}")
# --- CÓDIGO AÑADIDO PARA MOSTRAR RESULTADO EN PANTALLA ---
print("\n--- ¡RESULTADO DEL MODELO V1: AYATSIL! ---")
print(f"Campo Analizado: Ayatsil")
print(f"Versión del Modelo: simple_v1")
print(f"Error (RMSE): {rmse:.2f} mbd")
print("---------------------------------------------")
print(f"Interpretación: En promedio, las predicciones del modelo se equivocan en {rmse:.2f} mil barriles diarios.")


Evaluación completada. RMSE: 3.89

--- ¡RESULTADO DEL MODELO V1: AYATSIL! ---
Campo Analizado: Ayatsil
Versión del Modelo: simple_v1
Error (RMSE): 3.89 mbd
---------------------------------------------
Interpretación: En promedio, las predicciones del modelo se equivocan en 3.89 mil barriles diarios.


In [50]:
# --- NUEVO: GUARDAR ACTIVOS DEL MODELO ---

# 6. Guardar el Modelo Entrenado
os.makedirs(ruta_modelos, exist_ok=True)
ruta_modelo_salida = os.path.join(ruta_modelos, "ayatsil_xgb_simple_v1.json")
model.save_model(ruta_modelo_salida)
print(f"\nModelo guardado en: {ruta_modelo_salida}")


Modelo guardado en: C:\Users\elezr\Datalquemy\Proyectos\CNH\ml\modelos\ayatsil_xgb_simple_v1.json


In [52]:
# 7. Guardar la Evaluación del Modelo
os.makedirs(ruta_evaluacion, exist_ok=True)
ruta_evaluacion_salida = os.path.join(ruta_evaluacion, "eval_ayatsil_simple_v1.json")

info_evaluacion = {
    "nombre_modelo": "ayatsil_xgb_simple_v1",
    "fecha_entrenamiento": "2025-07-18",
    "metrica": {
        "tipo": "RMSE",
        "valor": round(rmse, 2)
    }
}

with open(ruta_evaluacion_salida, 'w') as f:
    json.dump(info_evaluacion, f, indent=4)

print(f"Evaluación guardada en: {ruta_evaluacion_salida}")

Evaluación guardada en: C:\Users\elezr\Datalquemy\Proyectos\CNH\ml\evaluacion\eval_ayatsil_simple_v1.json
